In [ ]:
# Add a custom RASTER file to the map: population of the World
from jeodpp import inter, imap
from ipywidgets import Layout

coll = inter.ImageCollection("SIMPLE")
p = coll.process()

minvalue =  0.0
maxvalue = 30.0
colorSchema = 'Spectral_soft'

p.parameter("file","/eos/jeodpp/data/base/Population/GLOBAL/WorldPop/VER1-0/Data/VRT/MOSAIC_ppp_prj_2020.vrt")
p.parameter("epsg", "4326")
p.parameter("nodata", "-99999.0")
p.parameter("scalemin", str(minvalue))
p.parameter("scalemax", str(maxvalue))
p.parameter("interpolate", 'nearest')  # 'bilinear'
p.parameter("colorscheme", colorSchema)

display(HTML(inter.colorSchemaLegend(colorSchema, Title="Population density per km2 in 2020", Markers=True, Height=32, MinValue=minvalue, MaxValue=maxvalue, NumPixels=700))) 

m = imap.Map(basemap=1, layout=Layout(height='600px'))
m.addLayer(p.toLayer())

inter.identifyPopup(m,p)

display(m)

In [ ]:
# Add a custom RASTER file to the map: EUCROPMAP (created by JRC D.5 unit)
from jeodpp import inter, imap
from ipywidgets import Layout
from ipyleaflet import WidgetControl

coll = inter.ImageCollection("SIMPLE")
eucropmap = coll.process()

outMap   = widgets.Output(layout=Layout(width='50%', height='750px'))
outChart = widgets.Output(layout=Layout(width='50%', height='750px'))


colormap = {  '0': '#000000',
            '211': '#a57000',
            '212': '#896054',
            '213': '#e2007c',
            '214': '#aa007c',
            '215': '#a05989',
            '216': '#ffd300',
            '217': '#00a8e2',
            '218': '#d69ebc',
            '219': '#d69ebc',
            '221': '#dda50a',
            '222': '#a800e2',
            '223': '#00af49',
            '230': '#00af49',
            '231': '#ffff00',
            '232': '#d1ff00',
            '233': '#267000',
            '240': '#f2a377',
            '250': '#e8bfff',
            '290': '#696969',
            '300': '#93cc93',
            '500': '#e8ffbf'
           }

valuemap = {  '0': 'No data',
            '211': 'Common wheat',
            '212': 'Durum wheat',
            '213': 'Barley',
            '214': 'Rye',
            '215': 'Oats',
            '216': 'Maize',
            '217': 'Rice',
            '218': 'Triticale',
            '219': 'Other cereals',
            '221': 'Potatoes',
            '222': 'Sugar beet',
            '223': 'Other root crops',
            '230': 'Other non permanent industrial crops',
            '231': 'Sunflower',
            '232': 'Rape and turnip rape',
            '233': 'Soya',
            '240': 'Dry pulses',
            '250': 'Fodder crops (cereals and leguminous)',
            '290': 'Bare arable land',
            '300': 'Woodland and Shrubland (incl. permanent crops)',
            '500': 'Grasslands'
           }

eucropmap.parameter("file","/eos/jeodpp/data/products/Landcover/EUROPE/EUCROPMAP/VER2018-0/Data/VRT/eucropmap_masked.vrt")
eucropmap.parameter("epsg", "3035")
eucropmap.parameter("nodata", "0")
eucropmap.parameter("scalemax", "-1")
eucropmap.parameter("interpolate", 'mode')
eucropmap.parameter("colormap",  str(colormap))
eucropmap.parameter("valuemap",  str(valuemap))

m = imap.Map(basemap=1, layout=Layout(height='730px'))
m.addLayer(eucropmap.toLayer())

with outMap:
    display(m)
    
inter.identifyPopup(m,eucropmap)

display(widgets.HBox([outMap,outChart]))

In [ ]:
# Custom python functions to calculate values occurrencies: img is a numpy array
def countOccurrencies(img):
    import math
    values = [0,211,212,213,214,215,216,217,218,219,221,222,223,230,231,232,233,240,250,290,300,500]

    counts = []
    for v in values:
        c = (img == v).sum()
        counts.append(c)
        
    return values, counts

In [ ]:
# Observe the map bounds and display the occurrencies using matplotlib at each zoom and pan
%matplotlib inline

def onBoundsChanged(args):
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt

    outChart.clear_output(wait=True)

    values, counts = eucropmap.extract(m,countOccurrencies)
    
    colors  = [colormap[str(x)] for x in values]
    labels = [valuemap[str(x)] for x in values]
    df = pd.DataFrame(zip(values,counts,labels), columns=['crop', 'pixels', 'labels'])

    with outChart:
        plt.rcParams.update({'font.size': 12})
        fig, ax = plt.subplots(figsize=(14,6.8))
        ax.set_title('Crop occurrencies')
        ax.bar(np.arange(len(df)), df['pixels'], color=colors, edgecolor='#999999')
        ax.set_xticks(np.arange(len(df)))
        ax.set_xticklabels(df['labels'], rotation=90)
        
        rects = ax.patches
        
        tot = np.sum(counts)
        percs = ["{:.1f}".format(100.0 * v / tot)+'%' for v in counts]
        percs = ['' if x=='0.0%' else x for x in percs]

        for rect, label in zip(rects, percs):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width() / 2, height + 5, label, ha='center', va='bottom', size=10)        
            
        plt.show()
        
m.observe(onBoundsChanged, 'bounds')
onBoundsChanged('')        

In [ ]:
im = inter.getWIDGETSimage('https://jeodpp.jrc.ec.europa.eu/services/shared/Notebooks/images/EUCROPMAP.png')
im.width = 420
legend = WidgetControl(widget=im, position='bottomright')
m.add_control(legend)